In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [2]:
import pinecone 

load_dotenv()
index_name = 'langchain-retrieval-agent-fast'

pinecone.init(      
	api_key=os.getenv("PINECONE_API_KEY"),      
	environment=os.getenv("PINECONE_ENVIRONMENT"),      
)      
index = pinecone.Index('langchain-retrieval-agent-fast')

/Users/ibrahimsaidi/Desktop/Builds/Kasuku/RefGPT/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone.from_existing_index(index_name, embeddings)

In [5]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 245}},
 'total_vector_count': 245}

### Memory

In [6]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [7]:
from langchain.chat_models import ChatOpenAI

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

In [9]:
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [10]:
question = "How many players can each team have on the field at once?"
result = qa({"question": question})

In [13]:
print(result)

{'question': 'How many players can each team have on the field at once?', 'chat_history': [HumanMessage(content='How many players can each team have on the field at once?', additional_kwargs={}, example=False), AIMessage(content='A match is played by two teams, each with a maximum of eleven players. However, a match may not start or continue if either team has fewer than seven players.', additional_kwargs={}, example=False)], 'answer': 'A match is played by two teams, each with a maximum of eleven players. However, a match may not start or continue if either team has fewer than seven players.'}


In [12]:
result['answer']

'A match is played by two teams, each with a maximum of eleven players. However, a match may not start or continue if either team has fewer than seven players.'